## In this notebook:

* 3 input dataframe: chr18, 19, 20
* using all numeric scores for predictors
* fully connected neural net
* training with dataloading and epochs
* aucuracy assessement of .954 on held-out test 

### Imports

In [88]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [89]:
#################################################### 
## pull in pickled dataframe:
df = pd.read_pickle("./pickled_df/all_scores_chr14,22.pkl")
df = df.replace({'-': 0})

# Split our data into two separate sets for training purposes
train, test = train_test_split(df, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                     SIFT_score SIFT4G_score Polyphen2_HDIV_score  \
chr pos      ref alt                                                
19  34394020 T   C            0            0                  1.0   
    34394032 A   G            0            0                  1.0   
    34394044 G   A            0            0                  1.0   
    34396362 C   G            0            0                    0   
    34399273 C   T            0            0                    0   

                     Polyphen2_HVAR_score LRT_score MutationTaster_score  \
chr pos      ref alt                                                       
19  34394020 T   C                      0  0.000000                    1   
    34394032 A   G                  0.998  0.000000                    1   
    34394044 G   A                  0.998  0.000000                    1   
    34396362 C   G                      0  0.000000                    1   
    34399273 C   T                      0  0.000034         

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [90]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(train)
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [91]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(27, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 27))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [92]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [0/11196 (0%)]	Loss: 0.696522
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [59/11196 (1%)]	Loss: 0.687396
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [118/11196 (1%)]	Loss: 0.682336
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [177/11196 (2%)]	Loss: 0.670819
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [236/11196 (2%)]	Loss: 0.652681
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [295/11196 (3%)]	Loss: 0.658385
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [354/11196 (3%)]	Loss: 0.633234
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [413/11196 (4%)]	Loss: 0.625456
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [472/11196 (4%)]	Loss: 0.564582
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [531/11196 (5%)]	Loss: 0.596271
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [590/11196 (5%)]	Loss: 0.603342
torch.Size([59, 27])
tor

 Train Epoch: 1/20 [5369/11196 (48%)]	Loss: 0.364524
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5428/11196 (48%)]	Loss: 0.402989
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5487/11196 (49%)]	Loss: 0.407569
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5546/11196 (49%)]	Loss: 0.376049
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5605/11196 (50%)]	Loss: 0.414322
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5664/11196 (51%)]	Loss: 0.429958
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5723/11196 (51%)]	Loss: 0.395637
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5782/11196 (52%)]	Loss: 0.400665
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5841/11196 (52%)]	Loss: 0.395348
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5900/11196 (53%)]	Loss: 0.400222
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [5959/11196 (53%)]	Loss: 0.420548
torch.Size([59, 27])
torch.Size([59])

 Train Epoch: 1/20 [10679/11196 (95%)]	Loss: 0.394709
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [10738/11196 (96%)]	Loss: 0.356676
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [10797/11196 (96%)]	Loss: 0.375802
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [10856/11196 (97%)]	Loss: 0.401431
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [10915/11196 (97%)]	Loss: 0.385418
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [10974/11196 (98%)]	Loss: 0.363755
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [11033/11196 (98%)]	Loss: 0.430227
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [11092/11196 (99%)]	Loss: 0.413928
torch.Size([45, 27])
torch.Size([45])
 Train Epoch: 1/20 [8505/11196 (99%)]	Loss: 0.338591
tensor([[0.0000, 0.0020, 1.0000,  ..., 0.7248, 0.7248, 0.7144],
        [0.0000, 0.0000, 1.0000,  ..., 0.7248, 0.7248, 0.7144],
        [0.0550, 0.1980, 1.0000,  ..., 0.7248, 0.7248, 0.7144],
        ...,
      

 Train Epoch: 2/20 [4425/11196 (39%)]	Loss: 0.348907
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4484/11196 (40%)]	Loss: 0.343819
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4543/11196 (41%)]	Loss: 0.369602
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4602/11196 (41%)]	Loss: 0.361745
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4661/11196 (42%)]	Loss: 0.416451
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4720/11196 (42%)]	Loss: 0.456503
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4779/11196 (43%)]	Loss: 0.401498
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4838/11196 (43%)]	Loss: 0.367759
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4897/11196 (44%)]	Loss: 0.382142
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [4956/11196 (44%)]	Loss: 0.421461
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [5015/11196 (45%)]	Loss: 0.378072
torch.Size([59, 27])
torch.Size([59])

 Train Epoch: 2/20 [9794/11196 (87%)]	Loss: 0.394036
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [9853/11196 (88%)]	Loss: 0.384519
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [9912/11196 (88%)]	Loss: 0.365930
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [9971/11196 (89%)]	Loss: 0.391253
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10030/11196 (89%)]	Loss: 0.412766
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10089/11196 (90%)]	Loss: 0.351620
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10148/11196 (91%)]	Loss: 0.390545
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10207/11196 (91%)]	Loss: 0.377351
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10266/11196 (92%)]	Loss: 0.349321
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10325/11196 (92%)]	Loss: 0.366221
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2/20 [10384/11196 (93%)]	Loss: 0.360099
torch.Size([59, 27])
torch.Siz

...

 Train Epoch: 18/20 [10738/11196 (96%)]	Loss: 0.349521
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 18/20 [10797/11196 (96%)]	Loss: 0.364162
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 18/20 [10856/11196 (97%)]	Loss: 0.355491
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 18/20 [10915/11196 (97%)]	Loss: 0.337579
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 18/20 [10974/11196 (98%)]	Loss: 0.347234
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 18/20 [11033/11196 (98%)]	Loss: 0.401795
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 18/20 [11092/11196 (99%)]	Loss: 0.350610
torch.Size([45, 27])
torch.Size([45])
 Train Epoch: 18/20 [8505/11196 (99%)]	Loss: 0.415831
tensor([[0.0000, 0.0020, 1.0000,  ..., 0.7248, 0.7248, 0.7144],
        [0.0000, 0.0000, 1.0000,  ..., 0.7248, 0.7248, 0.7144],
        [0.0550, 0.1980, 1.0000,  ..., 0.7248, 0.7248, 0.7144],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.7248, 0.7248, 0.7144],
        [0.0000, 0.0000, 0

 Train Epoch: 19/20 [4484/11196 (40%)]	Loss: 0.323859
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4543/11196 (41%)]	Loss: 0.401253
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4602/11196 (41%)]	Loss: 0.347913
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4661/11196 (42%)]	Loss: 0.345655
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4720/11196 (42%)]	Loss: 0.364592
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4779/11196 (43%)]	Loss: 0.396207
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4838/11196 (43%)]	Loss: 0.347693
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4897/11196 (44%)]	Loss: 0.393330
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [4956/11196 (44%)]	Loss: 0.385916
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [5015/11196 (45%)]	Loss: 0.327989
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [5074/11196 (45%)]	Loss: 0.350891
torch.Size([59, 27])
torch

 Train Epoch: 19/20 [9794/11196 (87%)]	Loss: 0.333142
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [9853/11196 (88%)]	Loss: 0.368816
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [9912/11196 (88%)]	Loss: 0.347394
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [9971/11196 (89%)]	Loss: 0.313447
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10030/11196 (89%)]	Loss: 0.373332
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10089/11196 (90%)]	Loss: 0.365817
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10148/11196 (91%)]	Loss: 0.372724
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10207/11196 (91%)]	Loss: 0.331130
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10266/11196 (92%)]	Loss: 0.347472
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10325/11196 (92%)]	Loss: 0.353382
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [10384/11196 (93%)]	Loss: 0.318155
torch.Size([59, 27]

 Train Epoch: 20/20 [3481/11196 (31%)]	Loss: 0.364113
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3540/11196 (32%)]	Loss: 0.388572
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3599/11196 (32%)]	Loss: 0.329421
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3658/11196 (33%)]	Loss: 0.313264
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3717/11196 (33%)]	Loss: 0.347160
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3776/11196 (34%)]	Loss: 0.329943
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3835/11196 (34%)]	Loss: 0.347160
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3894/11196 (35%)]	Loss: 0.374106
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [3953/11196 (35%)]	Loss: 0.364145
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [4012/11196 (36%)]	Loss: 0.363908
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [4071/11196 (36%)]	Loss: 0.352103
torch.Size([59, 27])
torch

 Train Epoch: 20/20 [8791/11196 (78%)]	Loss: 0.381058
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [8850/11196 (79%)]	Loss: 0.381349
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [8909/11196 (79%)]	Loss: 0.375541
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [8968/11196 (80%)]	Loss: 0.314895
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9027/11196 (81%)]	Loss: 0.347181
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9086/11196 (81%)]	Loss: 0.372064
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9145/11196 (82%)]	Loss: 0.330298
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9204/11196 (82%)]	Loss: 0.343237
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9263/11196 (83%)]	Loss: 0.319866
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9322/11196 (83%)]	Loss: 0.332640
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [9381/11196 (84%)]	Loss: 0.313315
torch.Size([59, 27])
torch

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [93]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]

y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))


 Testing Accuracy: 93.57%
